# Let's bring in the imports and the data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
sns.set_style('dark')
import sklearn
import tensorflow as tf
from tensorflow import keras

In [ ]:
data_path = '../input/dont-call-me-turkey/train.json'
test_path = '../input/dont-call-me-turkey/test.json'

df = pd.read_json(data_path)
test_df = pd.read_json(test_path)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
test_df.head()

In [ ]:
test_df.shape

# Data preparation

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.countplot(df['is_turkey'])
plt.tight_layout()

In [ ]:
length = df['audio_embedding'].apply(len)

y = length
x = np.arange(1, len(length)+1)

sns.countplot(length)
plt.tight_layout()

In [ ]:
plt.yscale('log')
sns.countplot(length)
plt.tight_layout()

In [ ]:
df.head()

In [ ]:
df['duration'] = df['end_time_seconds_youtube_clip'] = df['start_time_seconds_youtube_clip']
df.head()

In [ ]:
corr = df.corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5)
plt.tight_layout()

# Data preprocessing

In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 10

data = pad_sequences(df['audio_embedding'], maxlen=maxlen, padding='post')
labels = df['is_turkey'].values

In [ ]:
from keras.utils import to_categorical

train_size = int((80/100) * df.shape[0])

train_data = data[: train_size]
train_labels = to_categorical(labels[: train_size])

valid_data = data[train_size: ]
valid_labels = to_categorical(labels[train_size: ])

assert(len(train_data) == len(train_labels))
assert(len(valid_data) == len(valid_labels))

In [ ]:
train_data[0]

In [ ]:
num_features = len(train_data[0][0])
num_features

In [ ]:
train_labels[0]

In [ ]:
test_data = pad_sequences(test_df['audio_embedding'], maxlen=maxlen, padding='post')
test_data[0]

# Building and compiling the model

In [ ]:
def build_model():
    inp = keras.layers.Input(shape=(maxlen, num_features))
    x = keras.layers.BatchNormalization()(inp)
    
    x = keras.layers.Bidirectional( keras.layers.LSTM(128, return_sequences=True) )(x)
    x = keras.layers.Bidirectional( keras.layers.LSTM(64, return_sequences=True) )(x)
    
    avg_pool = keras.layers.GlobalAveragePooling1D()(x)
    max_pool = keras.layers.GlobalMaxPooling1D()(x)
    
    concat = keras.layers.concatenate([avg_pool, max_pool])
    
    hidden_1 = keras.layers.Dense(64)(concat)
    hidden_2 = keras.layers.Dropout(0.5)(hidden_1)
    hidden_activ = keras.layers.LeakyReLU()(hidden_2)
    
    output = keras.layers.Dense(2, activation="softmax")(hidden_activ)
    
    model = keras.Model(inputs=inp, outputs=output)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [ ]:
model = build_model()
keras.utils.plot_model(model, dpi=62)

# Training the model

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=2, verbose=1, min_lr=1e-8)

my_cb = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True)

history = model.fit(train_data, train_labels, batch_size=64, epochs=100, 
                    validation_data=(valid_data, valid_labels), callbacks=[reduce_lr, my_cb], verbose=2)

In [ ]:
epochs = len(history.history['loss'])
epochs

In [ ]:
y1 = history.history['loss']
y2 = history.history['val_loss']
x = np.arange(1, epochs+1)

plt.plot(x, y1, y2)
plt.legend(['loss', 'val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.tight_layout()

In [ ]:
y1 = history.history['acc']
y2 = history.history['val_acc']
x = np.arange(1, epochs+1)

plt.plot(x, y1, y2)
plt.legend(['acc', 'val_acc'])
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.tight_layout()

In [ ]:
model.evaluate(valid_data, valid_labels)

# Make predictions

In [ ]:
predictions = np.argmax(model.predict(test_data), axis=1)
predictions

In [ ]:
submission = pd.DataFrame({'vid_id': test_df['vid_id'], 'is_turkey': predictions})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)